In [1]:
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, MaxPooling1D, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, CSVLogger
import keras_metrics

from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
import numpy as np
import pickle

In [2]:
from cleaner import Cleaner 
cleaner = Cleaner()
cleaner.create_tokenizer_and_clean()

(95440, 14)
(95307, 14)
(143027, 14)
(135860, 14)
cleaning the text data 9
                            _id                   id            author_id  \
0      5eebee4181882c38d749ea78  1206599902098268160            396720801   
1      5ee8e22581882c38d70fc638  1238638476846923776            364528766   
2      5ef0334a81882c38d793e42d  1177136002714021888             47302521   
3      5ee7dfd981882c38d7f73f17  1253208996204421120             14436030   
4      5ee819e181882c38d7fd482e  1249774856985608192  1115538035167330304   
...                         ...                  ...                  ...   
47714  5ef8692181882c38d712840c   762215163495870464           1588108410   
47715  5ef8692281882c38d712841b   762095691002896384            141334191   
47716  5ef8692381882c38d712842b   762081066026934272             36214023   
47717  5ef8692781882c38d7128463   762434278814654464            425092162   
47719  5ef8692981882c38d712848d   762337525616771072   760560251435421696   



In [9]:
filename = "../data/extracted_tweets/large-bundle-clean.json"
df = pd.read_json(filename)
#df = self.clean_news(df)

y = df.about_suicide.values
sentences = df['text'].values
sentences.shape


(135860,)

In [10]:
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)
maxlen = 250
with open('../data/neural_network_config/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [11]:
filter_length = 300

        #model = Sequential()
#model.add(Embedding(vocab_size, 20, input_length=maxlen))
#model.add(Dropout(0.15))
#model.add(GlobalMaxPool1D())
#model.add(Dense(output_size, activation='sigmoid'))
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

model = Sequential()
model.add(Embedding(vocab_size, 20, input_length=maxlen))
model.add(Dropout(0.1))
model.add(Conv1D(filter_length, 5, activation='relu'))
model.add(Conv1D(filter_length, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(filter_length, 5, activation='relu'))
model.add(Conv1D(filter_length, 5, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall()])

In [ ]:
print("training model")
csv_logger = CSVLogger('log_loss.csv', append=False, separator=';')

callbacks = [
ModelCheckpoint(filepath='../data/neural_network_config/temp-model.h5', save_best_only=True), csv_logger]

history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=10,
                    validation_data=(X_test, y_test),
                    callbacks=callbacks)


training model
Epoch 1/10
Instructions for updating:
`inputs` is now automatically inferred
 1896/10190 [====>.........................] - ETA: 12:25 - loss: 0.0735 - accuracy: 0.9724 - precision: 0.9022 - recall: 0.7669        

In [6]:
print("saving model")
# serialize model to JSON
model_json = model.to_json()
with open("../data/neural_network_config/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../data/neural_network_config/model.h5")
print("Saved model to disk")

saving model
Saved model to disk


In [17]:
from keras.models import model_from_json
from cleaner import Cleaner

with open('../data/neural_network_config/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# load json and create model
json_file = open('../data/neural_network_config/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("../data/neural_network_config/model.h5")
print("Loaded model from disk")


# evaluate loaded model on test data
loaded_model.compile(optimizer='adam', loss='mean_squared_error',  metrics=['mae','accuracy'])

clean = Cleaner().clean_text

Loaded model from disk


In [29]:
sentence_test_no_suicide = [
    "Elegimos el hotel por su buena situación, en el centro, en la calle Ancha. Apenas a unos metros de la Catedral y el barrio Húmedo. En ese sentido, perfecto. El problema es que esa localización precisamente es lo que genera que sea imposible dormir. Nos tocó una habitación en el primer piso de la fachada que da a la calle Cervantes, repleta de bares. Hasta las 3 de la mañana con ruidos constantes, golpes, gritos. A partir de esa hora, borrachos gritando. No pudimos pegar ojo, algo que también fue culpa de la estrecha cama de 1,35 cm. Una auténtica pesadilla.  El hotel es un tres estrellas anclado en los finales de los ochenta. El baño y el mobiliario así lo demuestran. Las habitaciones están dispersas por varios edificios, pasillos y más pasillos, escaleras, etc...Caótico.  Otro aspecto negativo fue la televisión. Sólo tenía sintonizadas dos cadenas y no había posibilidad de resintonizar el aparato, ya que estaba bloqueado. Además, estaba contectada la descripción para ciegos, imposible de quitar, por lo que ver una película (lo único que se podía ver, la verdad), era toda una epopeya.  Al menos pudimos disfrutar de un circuito de spa gratuito. Se trata de un spa humilde, pero que no nos vino mal. SI hubiéramos sabido que íbamos a dormir así, nos quedamos en la piscina.  El desayuno no es buffet, es un café con zumo natural y una tostada.  Habíamos reservado aparcamiento, pero viendo las opiniones al respecto acerca de la estrechez e incomodidad, decidimos aparcar en uno público que está a 50 metros...al menos no tengo que volverme loco para entrar en una calle peatonal, maniobrar de locura y arriesgarme.  https://www.tripadvisor.es/ShowUserReviews-g187492-d8670166-r384189174-Hotel_Paris-Leon_Province_of_Leon_Castile_and_Leon.html#"
    "La comida está correcta, buena, pero un poco escasa. El precio adecuado si vas con promoción de El Tenedor. Aunque los camareros son agradables, el servicio es un poco caótico, con intervalos irregulares a la hora de traer la comida. Además, pedimos dos veces la carta para pedir más raciones, pero nunca la trajeron, y la cuenta venía con un error en el descuento aplicado",
    "Meh. Iba con unas expectativas medias y salí decepcionado. Es verdad que la carta pinta bien y que la están renovando (cambio de cocinero), pero no hay nada que sorprenda o deje un buen recuerdo. Sabores que intentan ser originales y acaban con el producto principal. Presentaciones muy similares, abusando de almendras, lechugas... Lo único destacable fueron los postres, especialmente un trampantojo de huevo. ",
    "que alegría que alboroto otro perrito piloto. muy correcto", "lo odio, fatal, que horror", "malo malísimo feo caca, horrible",
    "Recuerdo que antes decía, que en mí vivían sentimientos de agonía y tristeza, sentimientos que revivían con cada recuerdo que tenía, mi mente, un lugar abrumado, confundido, desolado y que pensaba que en ningún otro momento eso cambiaría, estaba tan lastimado no sólo por las personas, sino por la vida en general… De un momento a otro llega una pequeña casualidad, aquella que le daría un rumbo diferente a mi vida, que haría despertar aquel yo que se había ocultado por mucho tiempo, aquellos sentimientos que habían desaparecido, aquella pequeña casualidad, cobró mucha importancia, ya no sólo era una casualidad, era la razón de que yo no brincara al fondo de ese hoyo el cual me atraía cada vez más para hundirme dentro de el, era por quién en las mañanas ya no pensaba que todo era repetitivo, sino que todas las mañanas eran días nuevos, para poder vivir, poder disfrutar y sobre todo poder amar"
    ]
    
sentence_suicide = [
"Superar el duelo por la pérdida de algún ser querido durante el estado de alarma, abordar el llamado ‘síndrome de la cabaña’ o el miedo y la incertidumbre ante la vuelta al trabajo, gestionar el cierre de un negocio o una situación de desempleo... Durante estas últimas semanas, la población está afrontando complejas y muy difíciles situaciones que han reforzado nuestra preocupación por la salud física pero que también requieren de altas dosis de cuidados en el plano mental. Por esta razón, la Confederación Salud Mental España en colaboración con el Ministerio de Sanidad, Consumo y Bienestar acaba de lanzar la campaña Salud mental en la desescalada: cómo afrontar el largo camino, que tiene como objetivo ayudar a prevenir o bien abordar determinados momentos que tienen su origen en la crisis de la Covid-19 y que están afectando a gran número de personas. La campaña, difundida en redes sociales bajo el hasthag #DesescaladaSaludMental, aborda ocho escenarios principales donde cuidar la salud mental resulta prioritario y pone a disposición de la población una guía con recomendaciones para abordar dichas situaciones. Cómo afrontar el duelo Sin duda, una de las situaciones más dolorosa provocadas por la pandemia, que ha provocado que muchas personas no hayan podido despedirse de sus seres queridos ni compartir el dolor con su familia y amistades. “Tenemos que cuidar la muerte para cuidar la vida”, afirma el psicólogo Joseba Rico, subdirector de la Asociación madrileña de amigos y familiares de personas con esquizofrenia (AMAFE), que pertenece a la Federación Salud Mental Madrid. Mujer con mascarilla Ocho reglas de oro para aprender a gestionar la incertidumbre Según el especialista “los ritos son muy necesarios para cerrar la despedida”, y recuerda que durante estos meses se han producido situaciones muy diferentes: “Si por ejemplo alguien convive con sus padres y ha fallecido un abuelo, al menos hay una compañía porque les ha tocado vivir el confinamiento juntos, pero alguien que está en otra provincia, no ha podido estar cerca no ya de la persona que ha fallecido, sino de la familia. No ha podido recibir abrazos”. Por ello, Salud Mental España recomienda a todas las personas que pasan por un proceso de duelo permitirse sentir y exteriorizar la tristeza, crear rituales nuevos de despedida como escribir cartas de despedida o crear un collage con fotografías, reunirse con personas cercanas para realizar, cuando sea ya posible, una despedida u homenaje a la persona que ha fallecido y pedir ayuda profesional cuando se sienta que la situación nos supera. Miedo a salir de casa La psicóloga Alejandra Julio Berrio, colaboradora de Salud Mental España, explica que el llamado ‘Síndrome de la cabaña’ se produce cuando “relacionamos ‘casa’ con seguridad y ‘calle’ con peligro”. El hogar se convierte en una nueva zona de confort, hay miedo al contagio y salir supone un nuevo esfuerzo de adaptación. La experta aconseja “hacer trayectos emocionales cortos: si no te apetece salir a las ocho de la tarde a dar un paseo, puedes por ejemplo bajar la basura o ir a comprar el pan”. Además, se recomienda no forzar las salidas, crear rituales que nos hagan sentir más seguros (llevar mascarilla, uso de gel hidroalcohólico...), no compararse con nadie y ponerse pequeñas metas respetando el ritmo y los tiempos de cada uno. Personas con problemas de salud mental Durante el estado de alarma las personas que ya sufrían algún tipo de trastorno mental han podido sentirse más vulnerables al encontrarse mucho mas limitados los recursos necesarios para garantizar la continuidad de sus cuidados. “La clave es sentirte escuchado y acompañado y que, al menos puntualmente, en un momento dado seas importante para alguien que te presta su oído y su humanidad, para acoger toda la desesperación”, explica Joseba Rico. Entre las recomendaciones de la campaña se encuentra prestar atención a síntomas nuevos, observar si alguno ha empeorado y recordar que en esta situación es normal tener más estrés, pensamiento obsesivo, dificultad de concentración, estados de alerta y ansiedad. Mantener el autocuidado, respetar los horarios de sueño y cumplir el tratamiento y las recomendaciones del psicólogo o psiquiatra se hacen más necesarios que nunca.",
"Superar el duelo por la pérdida de algún ser querido durante el estado de alarma, abordar el llamado ‘síndrome de la cabaña’ o el miedo y la incertidumbre ante la vuelta al trabajo, gestionar el cierre de un negocio o una situación de desempleo... Durante estas últimas semanas, la población está afrontando complejas y muy difíciles situaciones que han reforzado nuestra preocupación por la salud física pero que también requieren de altas dosis de cuidados en el plano mental. Por esta razón, la Confederación Salud Mental España en colaboración con el Ministerio de Sanidad, Consumo y Bienestar acaba de lanzar la campaña Salud mental en la desescalada: cómo afrontar el largo camino, que tiene como objetivo ayudar a prevenir o bien abordar determinados momentos que tienen su origen en la crisis de la Covid-19 y que están afectando a gran número de personas. La campaña, difundida en redes sociales bajo el hasthag #DesescaladaSaludMental, aborda ocho escenarios principales donde cuidar la salud mental resulta prioritario y pone a disposición de la población una guía con recomendaciones para abordar dichas situaciones. Cómo afrontar el duelo Sin duda, una de las situaciones más dolorosa provocadas por la pandemia, que ha provocado que muchas personas no hayan podido despedirse de sus seres queridos ni compartir el dolor con su familia y amistades. “Tenemos que cuidar la muerte para cuidar la vida”, afirma el psicólogo Joseba Rico, subdirector de la Asociación madrileña de amigos y familiares de personas con esquizofrenia (AMAFE), que pertenece a la Federación Salud Mental Madrid. Mujer con mascarilla Ocho reglas de oro para aprender a gestionar la incertidumbre Según el especialista “los ritos son muy necesarios para cerrar la despedida”, y recuerda que durante estos meses se han producido situaciones muy diferentes: “Si por ejemplo alguien convive con sus padres y ha fallecido un abuelo, al menos hay una compañía porque les ha tocado vivir el confinamiento juntos, pero alguien que está en otra provincia, no ha podido estar cerca no ya de la persona que ha fallecido, sino de la familia. No ha podido recibir abrazos”. Por ello, Salud Mental España recomienda a todas las personas que pasan por un proceso de duelo permitirse sentir y exteriorizar la tristeza, crear rituales nuevos de despedida como escribir cartas de despedida o crear un collage con fotografías, reunirse con personas cercanas para realizar, cuando sea ya posible, una despedida u homenaje a la persona que ha fallecido y pedir ayuda profesional cuando se sienta que la situación nos supera. Miedo a salir de casa La psicóloga Alejandra Julio Berrio, colaboradora de Salud Mental España, explica que el llamado ‘Síndrome de la cabaña’ se produce cuando “relacionamos ‘casa’ con seguridad y ‘calle’ con peligro”. El hogar se convierte en una nueva zona de confort, hay miedo al contagio y salir supone un nuevo esfuerzo de adaptación. La experta aconseja “hacer trayectos emocionales cortos: si no te apetece salir a las ocho de la tarde a dar un paseo, puedes por ejemplo bajar la basura o ir a comprar el pan”. Además, se recomienda no forzar las salidas, crear rituales que nos hagan sentir más seguros (llevar mascarilla, uso de gel hidroalcohólico...), no compararse con nadie y ponerse pequeñas metas respetando el ritmo y los tiempos de cada uno. Personas con problemas de salud mental Durante el estado de alarma las personas que ya sufrían algún tipo de trastorno mental han podido sentirse más vulnerables al encontrarse mucho mas limitados los recursos necesarios para garantizar la continuidad de sus cuidados. “La clave es sentirte escuchado y acompañado y que, al menos puntualmente, en un momento dado seas importante para alguien que te presta su oído y su humanidad, para acoger toda la desesperación”, explica Joseba Rico. Entre las recomendaciones de la campaña se encuentra prestar atención a síntomas nuevos, observar si alguno ha empeorado y recordar que en esta situación es normal tener más estrés, pensamiento obsesivo, dificultad de concentración, estados de alerta y ansiedad. Mantener el autocuidado, respetar los horarios de sueño y cumplir el tratamiento y las recomendaciones del psicólogo o psiquiatra se hacen más necesarios que nunca. Prevención del suicidio Según datos de la Fundación ANAR, durante la crisis de la Covid-19 las llamadas de gente joven con ideas suicidas han subido del 1,9 al 8,3%. Esto demuestra que las situaciones de aislamiento, como las provocadas por el confinamiento, pueden provocar un aumento de las ideaciones suicidas. Salud Mental España recomienda contar con un “teléfono de auxilio” dentro de la red de apoyo (ya sea familia, amistades o profesional de la psicología) para llamar a esa persona en cualquier momento que se pueda necesitar; así como de entidades especializadas como la propia confederación o el Teléfono de la Esperanza. En la campaña se incide en que el primer paso es “reconocer que algo no está bien, ser consciente de que no se está solo o sola y compartir lo que se siente”.",
"Es evidente que el perro es el mejor amigo del hombre, pues son fieles y leales a sus dueños incluso cuando ya no están. Así lo demuestra este animal de Wuhan, China, que fue visto la semana pasada en el puente sobre el río Yangtsé esperando a su propietario, que tristemente suicidaron tirándose al río desde allí. Hichiko, el leal perro que esperaba todos los días a su dueño en el hospital. Un perro va todos los días al hospital desde hace tres meses a buscar a un dueño que no sabe que ha fallecido Los transeúntes que pasaban por allí veían al perro sin moverse, a veces tumbado, a veces mirando por los barrotes, así durante cuatro días. ADVERTISING Ads by Teads El pasado sábado, uno de los residentes de la zona quiso cogerlo para adoptarlo, pero el perro se escapó y, desde entonces, la Asociación de Protección de Animales Pequeños de Wuhan lo está buscando. Según declaró a Pear Video Du Fan, director de la protectora, la mascota siguió a su dueño hasta el puente sobre el río Yangtsé el pasado 1 de junio. Pedimos a los oficiales que comprobaran lo que había pasado exactamente. Estaba muy oscuro y no pudimos ver mucho en las cámaras de seguridad, pero vimos que la persona saltó, aseguró. Tal y como informó Daily Mail, la protectora tiene a sus voluntarios buscando al fiel animal y piden que cualquier persona que lo vea se presente en sus instalaciones para dar información. Todavía tenemos esperanza de encontrar un nuevo dueño para el perro, dijo Du Fan. Creemos que es muy triste que un perro tan leal se pierda en la calle.",
"El exfutbolista serbio Miljan Mrdakovic, antiguo jugador del Partizan, se quitó la vida el sábado en un piso en Belgrado con un tiro en la cabeza, informaron este domingo medios de Serbia. El diario Kurir señala que el exdelantero, que tenía 38 años, cometió el  en un piso alquilado en el que vivía con su novia. Mrdakovic comenzó su carrera en la ciudad meridional serbia de Nis, de donde pasó al Partizan. Militó en más de una docena de clubes, entre otros en el Anderlecht y el Gante belgas, el Salzburgo austríaco, el Maccabi israelí y los serbios Vojvodina, Rad y OFK Beograd, en el que disputó 48 partidos y marcó 24 tantos, y terminó su carrera, en 2017. Jugó nueve partidos con la selección serbia sub21 y tres encuentros con el equipo nacional en los Juegos Olímpicos de 2008. El diario Novosti indica que Mrdakovic tiene un hijo, de 16 años, que también se dedica al fútbol."
]


sentences = sentence_test_no_suicide + sentence_suicide

sentences = [clean(item) for item in sentences]


xnew = tokenizer.texts_to_sequences(sentences)
xnew = pad_sequences(xnew, padding='post', maxlen=maxlen)

ynew = loaded_model.predict(xnew)
print(ynew)

#print(ynew)
#print(multilabel_binarizer.inverse_transform(np.array([ynew])))

for i in range(0, len(ynew)):
    print("----")
    print(sentences[i])
    print(ynew[i] * 100)

[[8.0866725e-23]
 [2.2987961e-16]
 [6.6238595e-06]
 [1.9758940e-04]
 [4.5823753e-03]
 [1.7182910e-19]
 [4.6002132e-01]
 [1.0000000e+00]
 [1.0000000e+00]
 [9.9992323e-01]]
----
elegimos hotel buena situación centro calle ancha apenas metros catedral barrio húmedo perfecto problema localización precisamente genera imposible dormir tocó habitación primer piso fachada da calle cervantes repleta bares 3 mañana ruidos constantes golpes gritos partir hora borrachos gritando no pudimos pegar ojo culpa estrecha cama 1 35 cm auténtica pesadilla hotel tres estrellas anclado finales ochenta baño mobiliario así demuestran habitaciones dispersas varios edificios pasillos pasillos escaleras etccaótico aspecto negativo televisión sólo sintonizadas dos cadenas no posibilidad resintonizar aparato bloqueado además contectada descripción ciegos imposible quitar ver película único podía ver verdad toda epopeya menos pudimos disfrutar circuito spa gratuito trata spa humilde no vino mal si sabido íbamos dorm